In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('Data/data.csv')

FileNotFoundError: File b'Data/data.csv' does not exist

In [39]:
data = data.sort_values('time', ascending=False).reset_index()  # Se ordena la base de datos según fecha de terremoto


In [ ]:
data = data[['latitude']]

In [40]:
data = data.drop('index',1)                # Se eliminan las features time,magType,nst,gap,dmin,rms,net,id,
data = data.drop('time',1)                 # updated,place,type,horizontalError,depthError,magError,magNst,status,
data = data.drop('magType',1)              # locationSource y magSource, ya que escapan del análisis a realizar o
data = data.drop('nst',1)                  # tienen datos erróneos
data = data.drop('gap',1)
data = data.drop('dmin',1)
data = data.drop('rms',1)
data = data.drop('net',1)
data = data.drop('id',1)
data = data.drop('updated',1)
data = data.drop('place',1)
data = data.drop('type',1)
data = data.drop('horizontalError',1)
data = data.drop('depthError',1)
data = data.drop('magError',1)
data = data.drop('magNst',1)
data = data.drop('status',1)
data = data.drop('locationSource',1)
data = data.drop('magSource',1)

In [41]:
data

,latitude,longitude,depth,mag
0,37.6457,20.8702,10.00,4.7
1,65.2234,-151.6636,16.60,5.3
2,-22.4704,-68.6516,113.79,4.7
3,-56.0652,-27.4276,93.78,4.9
4,37.3128,20.4930,10.00,4.6
5,-19.3581,169.4061,207.24,4.5
6,-20.6365,-66.6667,236.32,4.8
7,40.1429,142.4098,50.28,5.0
8,18.3666,120.8819,10.00,4.6
9,37.2921,20.5582,10.00,4.3
